In [24]:
import os
import mlflow
import mlflow.pytorch
from dotenv import load_dotenv
import torch
import torch.nn as nn
import torch.nn.functional as F
import onnxruntime as ort

In [16]:
load_dotenv()

True

In [19]:
class TeacherWrapper(nn.Module):
    def __init__(self, onnx_path):
        super().__init__()
        self.onnx_path = onnx_path
        self.session = None
    
    def forward(self, x):
        if self.session is None:
            so = ort.SessionOptions()
            so.log_severity_level = 3
            self.session = ort.InferenceSession(
                self.onnx_path, 
                providers=['CUDAExecutionProvider', 'CPUExecutionProvider'], 
                sess_options=so
            )
        
        resized = F.interpolate(x, size=(112, 112), mode='bilinear', align_corners=False)
        input_name = self.session.get_inputs()[0].name
        ort_inputs = {input_name: resized.cpu().numpy()}
        ort_outputs = self.session.run(None, ort_inputs)
        embedding = torch.tensor(ort_outputs[0])
        return F.normalize(embedding, p=2, dim=1)

In [27]:
# Set environment variables untuk akses S3 dari luar container
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')  # Ambil dari .env
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')  # Ambil dari .env
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://127.0.0.1:9000'  # Akses dari luar container

In [28]:
# Upload to registry
mlflow.set_tracking_uri(uri='http://127.0.0.1:5000')
mlflow.set_experiment("Face Recognition Teacher Models")

with mlflow.start_run():
    teacher = TeacherWrapper('./glintr100.onnx')
    mlflow.pytorch.log_model(
        pytorch_model=teacher,
        artifact_path="teacher",
        registered_model_name="TeacherFaceRecognition"
    )

print("Teacher model uploaded to registry!")

MlflowException: API request to http://127.0.0.1:5000/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=Face+Recognition+Teacher+Models (Caused by ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))